# Práctica 1: Criptografía clásica
## UAM, 2022/2023

### Maitane Gómez González
### Ana Martínez Sabiote

## 1. Sustitución monoalfabeto

## 1.a Método afín
El siguiente programa implementa el método afín.

Llamada a la función:

afin {-C|-D} {-m |Zm|} {-a N×} {-b N+} [-i f ilein] [-o f ileout]


In [5]:
pip install gmpy2

Note: you may need to restart the kernel to use updated packages.


In [14]:
def algoritmo_euclides(a,b):
    if a%b == 0:
        return b
    else:
        return algoritmo_euclides(b, a%b)

In [15]:
mcd=algoritmo_euclides(150,39)
print(mcd)

3


In [16]:
def algoritmo_euclides_extendido(a,b):
    """
    # Condición a>b, sino las cambiamos
    if b>a:
        aux=a
        a=b
        b=aux
    """
    # Identidad de Bézout 1=u*a + v*b
    # El inverso de a módulo b es u. Recíprocamente, el inverso de b mod a es v
    if a==0:
        mcd=b
        u=0
        v=1
    else:
        mcd, x, y = algoritmo_euclides_extendido(b%a, a)
        u=y-(b//a)*x
        v=x
        
    return mcd, u, v

In [17]:
def inverso(a,m):
    result = algoritmo_euclides_extendido(a,m)
    # Comprobamos que el mcd es 1 para que exista inverso multiplicativo
    # En consecuencia, a y m determinan una función afín inyectiva
    if result[0] == 1:
        # Entonces devolvemos el coeficiente u (que acompaña a) de la Id. de Bézout
        return result[1]
    else:
        print("Error")

In [18]:
mcd=algoritmo_euclides_extendido(15,26)
print(mcd)

(1, 7, -4)


In [19]:
inv=inverso(15,26)
print(inv)

7
